# Amazon Web Scraping

### Installing required libraries

In [1]:
! pip install beautifulsoup4

In [2]:
! pip install requests

In [3]:
! pip install selenium

### Importing required libraries

In [4]:
# libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from requests.exceptions import RequestException

import warnings
warnings.filterwarnings('ignore')

### Web Scraping

In [7]:
# defining required fields
brand = []
description = []
price = []
link = []

# website url
url = 'https://www.amazon.in/'

try:
    
    # requesting response
    response = requests.get(url)
    response.raise_for_status()
    print('Web connection is fine --> response code:',response.status_code)
    
    # getting product category
    product_category = input('Enter the product category: ')
    
    # initializing browser
    driver = webdriver.Firefox()
    
    # running browser with url
    driver.get(url)
    
    for i in range(1,11):
        def search_url(product):
            url_template = 'https://www.amazon.in/s?k={}&page={}&crid=2TF1YHXH6RU5S&qid=1693134262&sprefix={}%2Caps%2C214&ref=sr_pg_{}'
    
            product = product.replace(' ', '+')
            return url_template.format(product, i, product, i)
        
        driver.get(search_url(product_category))
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # brand
        try:
            brand_ = soup.find_all('span', class_ = 'a-size-medium a-color-base a-text-normal')
            
            for i in brand_:
                try:
                    brand.append(i.text.split(' ')[0])
                except:
                    brand.append(None)
                    
        except AttributeError:
            print('Brand Attribute Not Found')
        
        # description
        try:
            des_ = soup.find_all('span', class_ = 'a-size-medium a-color-base a-text-normal')
            
            for i in des_:
                try:
                    description.append(i.text)
                except:
                    description.append(None)
                    
        except AttributeError:
            print('Description Attribute Not Found')
            
        # price
        try:
            price_ = soup.find_all('span', class_= 'a-price-whole')
            
            for i in price_:
                try:
                    price.append(i.text)
                except:
                    price.append(None)
                    
        except AttributeError:
            print('Price Attribute Not Found')

        # link
        try:
            link_ = soup.find_all('a', class_ = 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
            
            for i in link_:
                try:
                    link.append('https://www.amazon.in' + i.get('href'))
                except:
                    link.append(None)
                    
        except AttributeError:
            print('Link Attribute Not Found')
        

    driver.quit()
            
except RequestException as error:
    print(error)

Web connection is fine --> response code: 200
Enter the product category: smart phone


### Creating dataframe

In [8]:
# defining dataframe with the data 
df = pd.DataFrame()

df['Brand'] = brand
df['Description'] = description
df['Price'] = price
df['link'] = link
    
df.head()

,Brand,Description,Price,link
0,TECNO,"TECNO Pova 5 (Amber Gold, 8GB RAM,128GB Storag...","11,999",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,TECNO,"TECNO Pova 5 Pro 5G (Silver Fantasy, 8GB RAM,1...","14,999",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Redmi,"Redmi A2 (Aqua Blue, 2GB RAM, 32GB Storage) | ...","6,499",https://www.amazon.in/Redmi-Storage-Powerful-P...
3,OnePlus,"OnePlus Nord CE 3 Lite 5G (Chromatic Gray, 8GB...","21,999",https://www.amazon.in/OnePlus-Nord-Chromatic-2...
4,Samsung,"Samsung Galaxy M04 Light Green, 4GB RAM, 128GB...","9,499",https://www.amazon.in/Samsung-Storage-MediaTek...


In [9]:
df.shape

(200, 4)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Brand        200 non-null    object
 1   Description  200 non-null    object
 2   Price        200 non-null    object
 3   link         200 non-null    object
dtypes: object(4)
memory usage: 6.4+ KB


### Saving into csv file

In [11]:
# saving the dataset
df.to_csv('Amazon_' + product_category + '.csv')